In [11]:
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
import dgl
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense 
from keras.models import Model
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder

In [23]:
# !pip install --upgrade jupyter ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

Preprocess Node and Edge Features

In [14]:
artworks = pd.read_csv("/home/zhang/hajarlamtaai/KGC/PyG_data/artworks_augmented.csv")
properties = pd.read_csv("/home/zhang/hajarlamtaai/KGC/PyG_data/props.csv")

In [24]:
# node_features = np.arange(artworks.shape[0])
jid_encoder = LabelEncoder()
label_encoder = LabelEncoder()

artworks['JID_enc'] = jid_encoder.fit_transform(artworks['Jid'])
artworks['label_enc'] = label_encoder.fit_transform(artworks['label'])

node_features = artworks[['JID_enc', 'label_enc']].to_numpy()

prop_encoder = LabelEncoder()
properties['Prop_enc'] = prop_encoder.fit_transform(properties['Prop'])

edges_src = properties['Src'].to_numpy()
edges_dst = properties['Dest'].to_numpy()
edge_features = properties['Prop_enc'].to_numpy()


num_relations = len(prop_encoder.classes_)

num_nodes = artworks.shape[0]
adj_matrices = [sp.coo_matrix((np.ones(edges_src[edge_features == i].shape[0]),
                               (edges_src[edge_features == i], edges_dst[edge_features == i])),
                              shape=(num_nodes, num_nodes))
                for i in range(num_relations)]

node_features = node_features.astype('float32')
adj_matrices = [sp.csr_matrix(adj).astype('float32') for adj in adj_matrices]



In [26]:
#Compressed Sparse Row (CSR) : Coords(in dense adj matrix)	Values paires 
#Compressed Sparse column (CSC)
#Coordinate list (COO) [2,#existingedges]
adj_matrices

In [28]:
#convert to tf sparse tensors
tf_adj_matrices = [tf.sparse.SparseTensor(indices=np.column_stack((adj.nonzero())),
                                          values=adj.data,
                                          dense_shape=adj.shape)
                   for adj in adj_matrices]

In [37]:
# !ls relational-gcn/rgcn/layers/
%cd relational-gcn

/home/zhang/hajarlamtaai/KGC/relational-gcn


/home/zhang/miniconda3/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [47]:
from rgcn.layers.graph import GraphConvolution


In [52]:
# model Definition
input_features = Input(shape=(2,), dtype='float32')
input_adj = [Input(shape=(None, None), sparse=True) for _ in range(num_relations)]

graph_conv = GraphConvolution(output_dim=16, num_relations=num_relations)([input_features] + input_adj)
output = Dense(units=1, activation='sigmoid')(graph_conv)

model = Model(inputs=[input_features] + input_adj, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy')

NameError: name 'initializations' is not defined

In [ ]:
X_features = node_features  # Already in shape (num_nodes, 2)
X_adj = tf_adj_matrices

# Example labels (this should be replaced with actual labels)
y_train = np.random.randint(2, size=num_nodes)
y_val = np.random.randint(2, size=num_nodes)

# Fit the model
model.fit([X_features] + X_adj, y_train, validation_data=([X_features] + X_adj, y_val), epochs=10, batch_size=32)

Using StellarGraph implementation... 
https://github.com/stellargraph/stellargraph/blob/master/stellargraph/layer/gcn.py

In [43]:
%cd ..


/home/zhang/hajarlamtaai/KGC


/home/zhang/miniconda3/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


another take on DGL library ...


In [68]:
import pandas as pd
import torch
import dgl
from dgl.data import DGLDataset
import torch.nn as nn
from dgl.nn.pytorch import RelGraphConv


In [69]:
artworks = pd.read_csv("./PyG_data/artworks_augmented.csv")
properties = pd.read_csv("./PyG_data/props.csv")

In [70]:
artworks.columns

Index(['Id', 'Jid', 'entity', 'label'], dtype='object')

In [71]:
artworks["Jid"] = artworks["Jid"].astype('category').cat.codes
artwork_ids = artworks["Id"].tolist()
node_mapping = {artwork_id: idx for idx, artwork_id in enumerate(artwork_ids)}

# Create edge mappings
property_mapping = {prop: idx for idx, prop in enumerate(properties["Prop"].unique())}
properties["Prop_ID"] = properties["Prop"].map(property_mapping)

# Map edges to node indices
edges_src = properties["Src"].map(node_mapping).to_numpy()
edges_dst = properties["Dest"].map(node_mapping).to_numpy()
edge_features = properties["Prop_ID"].to_numpy()

In [72]:
class ArtworksDataset(DGLDataset):
    def __init__(self):
        super().__init__(name="Artworks")

    def process(self):
       
        node_ids = torch.arange(artworks.shape[0]).long().unsqueeze(1)
        node_jids = torch.from_numpy(artworks["Jid"].to_numpy()).long().unsqueeze(1)
        node_labels = torch.from_numpy(artworks["label"].astype('category').cat.codes.to_numpy()).long().unsqueeze(1)
        node_features = torch.cat([node_ids, node_jids, node_labels], dim=1)
        
        self.graph = dgl.graph((torch.from_numpy(edges_src), torch.from_numpy(edges_dst)), num_nodes=artworks.shape[0])
        # self.graph.ndata["id"] = node_features
        # self.graph.ndata["jid"] = node_jids
        # self.graph.ndata["label"] = node_labels
        self.graph.ndata["feat"] = node_features
        self.graph.edata["type"] = torch.from_numpy(edge_features).long()
        
        # Masks handling for link prediction task
        n_edges = properties.shape[0]
        n_train = int(n_edges * 0.6)
        n_val = int(n_edges * 0.2)
        train_mask = torch.zeros(n_edges, dtype=torch.bool)
        val_mask = torch.zeros(n_edges, dtype=torch.bool)
        test_mask = torch.zeros(n_edges, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.edata["train_mask"] = train_mask
        self.graph.edata["val_mask"] = val_mask
        self.graph.edata["test_mask"] = test_mask
        
    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

In [73]:
dataset = ArtworksDataset()
g = dataset[0]

In [74]:
g

Graph(num_nodes=32153, num_edges=217178,
      ndata_schemes={'feat': Scheme(shape=(3,), dtype=torch.int64)}
      edata_schemes={'type': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)})

In [75]:
# class RGCN(nn.Module):
#     def __init__(self, num_nodes, h_dim, num_rels):
#         super(RGCN, self).__init__()
#         self.embed_id = nn.Embedding(num_nodes, h_dim)
#         self.embed_jid = nn.Embedding(num_nodes, h_dim)
#         self.embed_label = nn.Embedding(num_nodes, h_dim)
#         self.conv1 = RelGraphConv(h_dim * 3, h_dim, num_rels, "basis", num_bases=5)
#         self.conv2 = RelGraphConv(h_dim, h_dim, num_rels, "basis", num_bases=5)

#     def forward(self, g, etypes):
#         h_id = self.embed_id(g.ndata['id'])
#         h_jid = self.embed_jid(g.ndata['jid'])
#         h_label = self.embed_label(g.ndata['label'])
#         h = torch.cat([h_id, h_jid, h_label], dim=1)
#         h = self.conv1(g, h, etypes)
#         h = torch.relu(h)
#         h = self.conv2(g, h, etypes)
#         return h

In [76]:
class RGCN(nn.Module):
    def __init__(self, num_nodes, input_dim, h_dim, num_rels):
        super(RGCN, self).__init__()
        self.conv1 = RelGraphConv(input_dim, h_dim, num_rels, "basis", num_bases=5)
        self.conv2 = RelGraphConv(h_dim, h_dim, num_rels, "basis", num_bases=5)

    def forward(self, g, etypes):
        h = g.ndata['feat']
        h = self.conv1(g, h, etypes)
        h = torch.relu(h)
        h = self.conv2(g, h, etypes)
        return h

#### can we import directrly a distmult decoder ( to kinda streamline the benchmark later???)

In [77]:
class DistMultDecoder(nn.Module):
    def __init__(self, h_dim, num_rels):
        super(DistMultDecoder, self).__init__()
        self.relation_emb = nn.Embedding(num_rels, h_dim)

    def forward(self, h, triplets):
        src, rel, dst = triplets[:, 0], triplets[:, 1], triplets[:, 2]
        src_emb = h[src]
        rel_emb = self.relation_emb(rel)
        dst_emb = h[dst]
        score = (src_emb * rel_emb * dst_emb).sum(dim=1)
        return score

In [92]:
num_nodes = g.number_of_nodes()
input_dim = 3  # id, jid, label
num_rels = len(property_mapping)

rgcn_model = RGCN(num_nodes, input_dim, 16, num_rels)
decoder = DistMultDecoder(16, num_rels)

In [93]:
# move tensors to GPU (not working)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# g.ndata['feat'] = g.ndata['feat'].to(device)
# g.edata['type'] = g.edata['type'].to(device)
# g.edata['train_mask'] = g.edata['train_mask'].to(device)
# g.edata['val_mask'] = g.edata['val_mask'].to(device)
# g.edata['test_mask'] = g.edata['test_mask'].to(device)


In [94]:
def get_triplets(g, mask):
    edges_src, edges_dst = g.edges()
    src = edges_src[mask]
    dst = edges_dst[mask]
    rel = g.edata['type'][mask]
    return torch.stack((src, rel, dst), dim=1)

In [95]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.size(0)), torch.zeros(neg_score.size(0))])
    return nn.BCEWithLogitsLoss()(scores, labels)

In [96]:
optimizer = torch.optim.Adam(list(rgcn_model.parameters()) + list(decoder.parameters()), lr=0.01)


In [97]:
for epoch in range(500):
    rgcn_model.train()
    h = rgcn_model(g, g.edata['type'])
    pos_triplets = get_triplets(g, g.edata['train_mask'])
    #generate negative samples by corupting the relation
    neg_triplets = torch.stack((pos_triplets[:, 0], torch.randint(0, num_rels, (pos_triplets.size(0),)), pos_triplets[:, 2]), dim=1) 
    pos_score = decoder(h, pos_triplets)
    neg_score = decoder(h, neg_triplets)

    loss = compute_loss(pos_score, neg_score)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        rgcn_model.eval()
        with torch.no_grad():
            val_triplets = get_triplets(g, g.edata['val_mask'])
            val_neg_triplets = torch.stack((val_triplets[:, 0], torch.randint(0, num_rels, (val_triplets.size(0),)), val_triplets[:, 2]), dim=1)
            val_pos_score = decoder(h, val_triplets)
            val_neg_score = decoder(h, val_neg_triplets)
            val_loss = compute_loss(val_pos_score, val_neg_score)
        print(f'Epoch {epoch} | Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}')
        


RuntimeError: expected m1 and m2 to have the same dtype, but got: long int != float

In [30]:
# Testing
rgcn_model.eval()
with torch.no_grad():
    h = rgcn_model(g, g.edata['type'])
    test_triplets = get_triplets(g, g.edata['test_mask'])
    test_neg_triplets = torch.stack((test_triplets[:, 0], torch.randint(0, num_rels, (test_triplets.size(0),)), test_triplets[:, 2]), dim=1)
    test_pos_score = decoder(h, test_triplets)
    test_neg_score = decoder(h, test_neg_triplets)
    test_loss = compute_loss(test_pos_score, test_neg_score)
print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 1.1646


In [34]:
def query_model(h, src, rel=None, dst=None):
    if rel is not None:  # Query (h, r, ?)
        src = torch.tensor([src], device=h.device)  
        rel = torch.tensor([rel], device=h.device) 
        src_emb = h[src]
        rel_emb = decoder.relation_emb(rel)
        scores = (src_emb * rel_emb * h).sum(dim=1)
        return scores.argsort(descending=True)
    elif dst is not None:  # Query (h, ?, t)
        dst = torch.tensor([dst], device=h.device) 
        dst_emb = h[dst]
        scores = (h * dst_emb).sum(dim=1)
        return scores.argsort(descending=True)

In [43]:
h = rgcn_model(g, g.edata['type'])
src_node = 14203	
rel_id = 11
k = 10
top_k = query_model(h, src_node, rel=rel_id)[:k]
print(f'Top 10 target nodes for ({src_node}, {rel_id}, ?): {top_k.tolist()}')

dst_node = 5867
top_k_rel = query_model(h, src_node, dst=dst_node)[:k]
print(f'Top 10 relations for ({src_node}, ?, {dst_node}): {top_k_rel.tolist()}')

Top 10 target nodes for (14203, 11, ?): [12107, 29524, 24271, 5461, 1801, 1947, 5591, 24907, 13969, 22563]
Top 10 relations for (14203, ?, 5867): [18507, 4642, 5102, 25067, 11245, 1157, 21816, 16012, 11600, 28626]


In [50]:
def compute_ranks(h, triplets, model, num_nodes):
    ranks = []
    for i in range(triplets.shape[0]):
        src, rel, dst = triplets[i]
        
        # Compute scores for all possible targets
        src_emb = h[src].unsqueeze(0).repeat(num_nodes, 1) #(num_nodes , embedding_dim)
        rel_emb = model.relation_emb(rel).unsqueeze(0).repeat(num_nodes, 1)  #(num_nodes , embedding_dim)
        dst_emb = h #all node embeddings as possible destination embeddings
        
        scores = (src_emb * rel_emb * dst_emb).sum(dim=1)
        
        # Rank the correct target
        target_score = scores[dst].item()
        rank = (scores > target_score).sum().item() + 1
        ranks.append(rank)
    
    return ranks


In [51]:
def evaluate_ranks(ranks):
    mr = sum(ranks) / len(ranks)
    mrr = sum(1.0 / rank for rank in ranks) / len(ranks)
    hits_at_10 = sum(rank <= 10 for rank in ranks) / len(ranks)
    hits_at_5 = sum(rank <= 5 for rank in ranks) / len(ranks)
    hits_at_1 = sum(rank == 1 for rank in ranks) / len(ranks)
    
    return mr, mrr, hits_at_10, hits_at_5, hits_at_1


In [56]:
# Testing with rank-based metrics
rgcn_model.eval()
with torch.no_grad():
    h = rgcn_model(g, g.edata['type'])
    test_triplets = get_triplets(g, g.edata['test_mask'])
    ranks = compute_ranks(h, test_triplets, decoder, g.number_of_nodes())
    
    mr, mrr, hits_at_10, hits_at_5, hits_at_1 = evaluate_ranks(ranks)
    
    print(f'Test MR: {mr:.4f}')
    print(f'Test MRR: {mrr:.4f}')
    print(f'Test Hits@10: {hits_at_10:.4f}')
    print(f'Test Hits@5: {hits_at_5:.4f}')
    print(f'Test Hits@1: {hits_at_1:.4f}')


Test MR: 3914.4604
Test MRR: 0.1389
Test Hits@10: 0.2941
Test Hits@5: 0.2091
Test Hits@1: 0.0693
